In [ ]:
import torch
from torchvision import models, datasets, transforms
from torch import nn, optim
from torch.utils.data import DataLoader, random_split



# Define transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Dataset directory
data_dir = './dataset'

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Calculate split sizes
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Define DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)


In [ ]:
# Load a pretrained ResNet model
model = models.resnet18(pretrained=True)
# Modify the final layer for classification
num_classes = len(dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Model Training

In [2]:
# Training and validation
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {running_loss/len(train_loader):.4f}, "
          f"Val Loss: {val_loss/len(val_loader):.4f}, "
          f"Val Accuracy: {correct/total:.4f}")


Epoch 1/10, Train Loss: 2.6426, Val Loss: 5.9393, Val Accuracy: 0.0970
Epoch 2/10, Train Loss: 1.2728, Val Loss: 3.2487, Val Accuracy: 0.3818
Epoch 3/10, Train Loss: 0.6144, Val Loss: 2.4907, Val Accuracy: 0.3879
Epoch 4/10, Train Loss: 0.3026, Val Loss: 3.0118, Val Accuracy: 0.4242
Epoch 5/10, Train Loss: 0.1796, Val Loss: 1.9895, Val Accuracy: 0.5758
Epoch 6/10, Train Loss: 0.0645, Val Loss: 1.6549, Val Accuracy: 0.5879
Epoch 7/10, Train Loss: 0.0594, Val Loss: 2.0861, Val Accuracy: 0.5515
Epoch 8/10, Train Loss: 0.0548, Val Loss: 2.0961, Val Accuracy: 0.5515
Epoch 9/10, Train Loss: 0.0695, Val Loss: 2.3544, Val Accuracy: 0.4727
Epoch 10/10, Train Loss: 0.0944, Val Loss: 1.7925, Val Accuracy: 0.5697
Training Accuracy: 98.39%
Validation Accuracy: 56.97%
Testing Accuracy: 53.29%


# Model saving

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'res_crop_classifier_cnn.pth')

# Checking accuracy

In [ ]:
# Accuracy calculation function
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculations for evaluation
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Get predictions
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # Get the index of the max probability
            
            # Update counts
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total * 100  # Convert to percentage
    return accuracy


In [ ]:
# Calculate accuracies for train, validation, and test sets
train_accuracy = calculate_accuracy(model, train_loader, device)
val_accuracy = calculate_accuracy(model, val_loader, device)
test_accuracy = calculate_accuracy(model, test_loader, device)

# Print the accuracies
print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Validation Accuracy: {val_accuracy:.2f}%")
print(f"Testing Accuracy: {test_accuracy:.2f}%")


# Model testing

In [1]:
from PIL import Image
import torch
from torchvision import transforms, models

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the saved model
model = models.resnet18(pretrained=False)  # Recreate ResNet18 architecture
num_classes = len(dataset.classes)  # Number of classes
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)  # Modify the final layer
model.load_state_dict(torch.load('final_model.pth'))  # Load weights
model = model.to(device)
model.eval()  # Set the model to evaluation mode

# Define image transformations (must match the training preprocessing)
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

# Load and preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Ensure 3 channels (RGB)
    image = transform(image)  # Apply transformations
    image = image.unsqueeze(0)  # Add batch dimension (1, 3, 224, 224)
    return image

# Predict function
def predict(image_path, model, device, class_names):
    image = preprocess_image(image_path).to(device)  # Preprocess and move to device
    with torch.no_grad():
        outputs = model(image)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Get class index with highest score
    return class_names[predicted.item()]  # Return class name

# Class names from dataset
class_names = dataset.classes

# Test the model on an example image
image_path = 'test.jpg'  # Replace with your image path
predicted_class = predict(image_path, model, device, class_names)

print(f"Predicted class: {predicted_class}")


ModuleNotFoundError: No module named 'torch'